# Del 2 – Fördjupning och pipeline

Jag har organiserat analysen från Del 1 med funktioner och klassen HealthAnalyzer. jag använder linjär regression och PCA för att hitta mönster i datan.


In [ ]:
# Importera bibliotek
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Importera egna moduler
from src.data_loader import load_data
from src.analysis import descriptive_stats, plot_histogram, plot_boxplot_by_sex, plot_disease_by_sex
from src.models import HealthAnalyzer

np.random.seed(42)


## Ladda data

läser in hälsodatamängden från CSV och visar de första raderna.


In [ ]:
df = load_data("data/health_study_dataset.csv")
df.head(3)


## Beskrivande statistik

Beräkning av medelvärde, median, min och max för viktiga kolumner.


In [ ]:
columns = ["age", "weight", "height", "systolic_bp", "cholesterol"]
stats = descriptive_stats(df, columns)
print(stats)


## Grafer

### Histogram över blodtryck


In [ ]:
plot_histogram(df, "systolic_bp", "Histogram över blodtryck", "Systoliskt blodtryck")


### Boxplot över vikt per kön


In [ ]:
plot_boxplot_by_sex(df, "weight")


### Andel med sjukdom per kön


In [ ]:
plot_disease_by_sex(df)


## HealthAnalyzer

jag använder HealthAnalyzer för att räkna andelen sjuka, beräkna konfidensintervall och göra linjär regression.


In [ ]:
analyzer = HealthAnalyzer(df)

# Andel med sjukdom
disease_prop = analyzer.calculate_disease_proportion()
print(f"Andel personer med sjukdom: {disease_prop:.2f}")

# Bootstrap CI för blodtryck
ci_lower, ci_upper = analyzer.bootstrap_ci("systolic_bp")
print(f"95% bootstrap-konfidensintervall: ({ci_lower:.2f}, {ci_upper:.2f})")

# Linjär regression: blodtryck ~ ålder + vikt
coef, intercept = analyzer.linear_regression(["age", "weight"], "systolic_bp")
print(f"Regression: blodtryck = {intercept:.2f} + {coef[0]:.2f}*ålder + {coef[1]:.2f}*vikt")


## Utökad analys

### Blodtryck vs Ålder

Visualisering av sambandet mellan ålder och systoliskt blodtryck.


In [ ]:
plt.scatter(df["age"], df["systolic_bp"], alpha=0.6)
plt.title("Blodtryck vs Ålder")
plt.xlabel("Ålder")
plt.ylabel("Systoliskt blodtryck")
plt.show()


### PCA – hitta mönster i data

jag använder PCA för att minska antalet variabler och hitta mönster i hälsodatan.


In [ ]:
# Väljer kolumner som är numeriska
features = ["age", "weight", "height", "systolic_bp", "cholesterol"]
X = df[features].values

# Standardisera data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(8,5))
plt.scatter(X_pca[:,0], X_pca[:,1], alpha=0.6)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("PCA av hälsodata (2 komponenter)")
plt.show()

# Förklaring av varians
explained_variance = pca.explained_variance_ratio_
print(f"Förklarad varians: PC1={explained_variance[0]:.2f}, PC2={explained_variance[1]:.2f}")


### RESULTAT

- Totalt hade 6 % av deltagarna sjukdomen, med en högre andel bland män än kvinnor. Blodtrycket låg inom ett bootstrap-intervall på 148,31–150,10, och både ålder och vikt var positivt kopplade till blodtryck. PCA visade att två komponenter förklarade 67 % av variationen i datan.